In [ ]:
!pip install -q ultralytics pycocotools opencv-python-headless matplotlib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# The path now starts with /content/drive/MyDrive/
!unzip "/content/drive/MyDrive/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation.zip" -d /content/SorghumWeedDataset_Segmentation

Archive:  /content/drive/MyDrive/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation.zip
  inflating: /content/SorghumWeedDataset_Segmentation/ReadMe.txt  
   creating: /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Test/
  inflating: /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Test/TestSorghumWeed (1).JPG  
  inflating: /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Test/TestSorghumWeed (10).JPG  
  inflating: /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Test/TestSorghumWeed (11).JPG  
  inflating: /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Test/TestSorghumWeed (12).JPG  
  inflating: /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Test/TestSorghumWeed (13).JPG  
  inflating: /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Test/TestSorghumWeed (14).JPG  
  inflating: /content/SorghumWeed

In [ ]:
#Conversion of coco.json to .txt file used in YOLOv8
import os
import json
from pathlib import Path
from pycocotools.coco import COCO
import numpy as np
from tqdm import tqdm

# This function will handle the conversion
def convert_coco_to_yolo_segmentation(json_path, save_dir):
    # Check if the JSON file exists before proceeding
    if not os.path.exists(json_path):
        print(f"WARNING: Annotation file not found at {json_path}. Skipping this set.")
        return

    os.makedirs(save_dir, exist_ok=True)

    # Initialize COCO api for instance annotations
    coco = COCO(json_path)

    # Create a mapping from COCO category IDs to a continuous range (0, 1, 2...)
    cat_ids = coco.getCatIds()
    categories = coco.loadCats(cat_ids)
    categories.sort(key=lambda x: x['id'])

    cat_id_to_yolo_id = {cat['id']: i for i, cat in enumerate(categories)}

    print(f"\nConverting {json_path}...")
    print(f"Category mapping: {cat_id_to_yolo_id}")

    # Get all images
    img_ids = coco.getImgIds()

    for img_id in tqdm(img_ids, desc=f"Processing {Path(json_path).parent.name} images"):
        img_info = coco.loadImgs(img_id)[0]

        file_name = Path(img_info['file_name']).stem
        height, width = img_info['height'], img_info['width']

        label_path = os.path.join(save_dir, f"{file_name}.txt")

        with open(label_path, 'w') as f:
            ann_ids = coco.getAnnIds(imgIds=img_id, iscrowd=None)
            anns = coco.loadAnns(ann_ids)

            for ann in anns:
                if 'segmentation' in ann and len(ann['segmentation']) > 0:
                    yolo_class_id = cat_id_to_yolo_id[ann['category_id']]

                    for seg in ann['segmentation']:
                        # Normalize coordinates
                        normalized_seg = []
                        for i in range(0, len(seg), 2):
                            x = seg[i] / width
                            y = seg[i+1] / height
                            normalized_seg.extend([x, y])

                        # Format for YOLO: class_id x1 y1 x2 y2 ...
                        segment_str = " ".join(f"{coord:.6f}" for coord in normalized_seg)
                        f.write(f"{yolo_class_id} {segment_str}\n")

# --- PATHS---

# Base path where your unzipped data is
data_base_path = "/content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation"

# Path where the new YOLO formatted dataset will be created
yolo_dataset_path = "/content/yolo_dataset"

# --- FILENAMES (VERIFIED FROM YOUR OUTPUT) ---
train_json_file = "TrainSorghumWeed_coco.json"
val_json_file = "ValidateSorghumWeed_coco.json"
test_json_file = "TestSorghumWeed_coco.json"
# ---------------------------------------------

# Create the necessary directory structure for YOLO
print("Creating new YOLO dataset structure...")
os.makedirs(os.path.join(yolo_dataset_path, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_dataset_path, "images/val"), exist_ok=True)
os.makedirs(os.path.join(yolo_dataset_path, "images/test"), exist_ok=True)
os.makedirs(os.path.join(yolo_dataset_path, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_dataset_path, "labels/val"), exist_ok=True)
os.makedirs(os.path.join(yolo_dataset_path, "labels/test"), exist_ok=True)

# Copy images to the new structure (using corrected Title Case folder names)
print("Copying images...")
!cp -n {data_base_path}/Train/*.JPG {yolo_dataset_path}/images/train/
!cp -n {data_base_path}/Validate/*.JPG {yolo_dataset_path}/images/val/
!cp -n {data_base_path}/Test/*.JPG {yolo_dataset_path}/images/test/

# Run the conversion for each set (using corrected Title Case folder names)
convert_coco_to_yolo_segmentation(
    json_path=os.path.join(data_base_path, "Train", train_json_file),
    save_dir=os.path.join(yolo_dataset_path, "labels/train")
)
convert_coco_to_yolo_segmentation(
    json_path=os.path.join(data_base_path, "Validate", val_json_file),
    save_dir=os.path.join(yolo_dataset_path, "labels/val")
)
convert_coco_to_yolo_segmentation(
    json_path=os.path.join(data_base_path, "Test", test_json_file),
    save_dir=os.path.join(yolo_dataset_path, "labels/test")
)

print("\nConversion Complete!")
print(f"Your YOLO-formatted dataset is ready at: {yolo_dataset_path}")

Creating new YOLO dataset structure...
Copying images...
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!

Converting /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Train/TrainSorghumWeed_coco.json...
Category mapping: {1: 0, 2: 1, 3: 2}


Processing Train images: 100%|██████████| 202/202 [00:00<00:00, 1652.95it/s]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Converting /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Validate/ValidateSorghumWeed_coco.json...
Category mapping: {1: 0, 2: 1, 3: 2}


Processing Validate images: 100%|██████████| 25/25 [00:00<00:00, 1947.18it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Converting /content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Test/TestSorghumWeed_coco.json...
Category mapping: {1: 0, 2: 1, 3: 2}


Processing Test images: 100%|██████████| 25/25 [00:00<00:00, 1944.47it/s]


Conversion Complete!
Your YOLO-formatted dataset is ready at: /content/yolo_dataset


In [ ]:
# This command lists all files and folders recursively
!ls -R /content/SorghumWeedDataset_Segmentation/

/content/SorghumWeedDataset_Segmentation/:
ReadMe.txt  SorghumWeedDataset_Segmentation

/content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation:
Test  Train  Validate

/content/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation/Test:
'TestSorghumWeed (10).JPG'  'TestSorghumWeed (23).JPG'
'TestSorghumWeed (11).JPG'  'TestSorghumWeed (24).JPG'
'TestSorghumWeed (12).JPG'  'TestSorghumWeed (25).JPG'
'TestSorghumWeed (13).JPG'  'TestSorghumWeed (2).JPG'
'TestSorghumWeed (14).JPG'  'TestSorghumWeed (3).JPG'
'TestSorghumWeed (15).JPG'  'TestSorghumWeed (4).JPG'
'TestSorghumWeed (16).JPG'  'TestSorghumWeed (5).JPG'
'TestSorghumWeed (17).JPG'  'TestSorghumWeed (6).JPG'
'TestSorghumWeed (18).JPG'  'TestSorghumWeed (7).JPG'
'TestSorghumWeed (19).JPG'  'TestSorghumWeed (8).JPG'
'TestSorghumWeed (1).JPG'   'TestSorghumWeed (9).JPG'
'TestSorghumWeed (20).JPG'   TestSorghumWeed_coco.json
'TestSorghumWeed (21).JPG'   TestSorghumWeed_csv.csv
'TestSorghumWeed (22).JPG'   Te

In [ ]:
import yaml

class_names = ['Sorghum', 'Grasses', 'Broad-leaf weeds']

# Create the dictionary for the YAML file
dataset_config = {
    'path': '/content/yolo_dataset',  # The root directory of your dataset
    'train': 'images/train',         # Path to train images (relative to 'path')
    'val': 'images/val',             # Path to validation images (relative to 'path')
    'test': 'images/test',           # Path to test images (relative to 'path')
    'nc': len(class_names),          # Number of classes (3)
    'names': class_names             # List of class names
}

# Write the dictionary to a dataset.yaml file
yaml_file_path = '/content/yolo_dataset/dataset.yaml'
with open(yaml_file_path, 'w') as file:
    yaml.dump(dataset_config, file, default_flow_style=False)

print(f"dataset.yaml created successfully at {yaml_file_path}")
print("\nFile contents:")
with open(yaml_file_path, 'r') as file:
    print(file.read())

dataset.yaml created successfully at /content/yolo_dataset/dataset.yaml

File contents:
names:
- Sorghum
- Grasses
- Broad-leaf weeds
nc: 3
path: /content/yolo_dataset
test: images/test
train: images/train
val: images/val



In [ ]:
!yolo task=segment mode=train model=yolov8s-seg.pt data='/content/yolo_dataset/dataset.yaml' epochs=50 imgsz=640 batch=8

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

In [ ]:
#prediction
!yolo task=segment mode=predict model='/content/runs/segment/train2/weights/best.pt' source='/content/yolo_dataset/images/test/'

In [ ]:
#evaluation
!yolo task=segment mode=val model='/content/runs/segment/train2/weights/best.pt' data='/content/yolo_dataset/dataset.yaml'

In [ ]:
# ===================================================================
# FINAL SCRIPT TO CREATE YOUR PERMANENT, PROCESSED DATASET
# ===================================================================
import os
import yaml
from google.colab import drive
from pathlib import Path
from pycocotools.coco import COCO
from tqdm import tqdm

# 1. Mount Google Drive
print("STEP 1: Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

# 2. Define Permanent Paths in Your Drive
project_drive_path = "/content/drive/MyDrive/Sorghum_Project"

# ⚡️ THIS IS THE CORRECTED, DEEPER PATH TO YOUR ZIP FILE ⚡️
source_zip_path = os.path.join(project_drive_path, "source_dataset/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation.zip")

processed_dataset_path = os.path.join(project_drive_path, "yolo_dataset")
temp_unzip_path = "/content/temp_unzipped"

# 3. Unzip from the correct path
print(f"\nSTEP 2: Unzipping source data from {source_zip_path}...")
# Check if zip file exists before unzipping
if not os.path.exists(source_zip_path):
    print(f"❌ ERROR: Cannot find the zip file at the specified path. Please check your folder structure in Google Drive again.")
else:
    !unzip -q "{source_zip_path}" -d {temp_unzip_path}
    print("✅ Unzip complete.")

    # 4. Run the Conversion and save directly to Drive
    print("\nSTEP 3: Converting annotations and saving to Drive...")
    def convert_coco_to_yolo_segmentation(json_path, save_dir):
        if not os.path.exists(json_path):
            print(f"⚠️  WARNING: Could not find {json_path}. Skipping.")
            return
        os.makedirs(save_dir, exist_ok=True)
        coco = COCO(json_path)
        cat_ids = coco.getCatIds()
        categories = coco.loadCats(cat_ids)
        categories.sort(key=lambda x: x['id'])
        cat_id_to_yolo_id = {cat['id']: i for i, cat in enumerate(categories)}
        for img_id in tqdm(coco.getImgIds(), desc=f"  -> Processing {Path(json_path).parent.name}"):
            img_info = coco.loadImgs(img_id)[0]
            file_name = Path(img_info['file_name']).stem
            height, width = img_info['height'], img_info['width']
            label_path = os.path.join(save_dir, f"{file_name}.txt")
            with open(label_path, 'w') as f:
                for ann in coco.loadAnns(coco.getAnnIds(imgIds=img_id)):
                    if 'segmentation' in ann and len(ann['segmentation']) > 0:
                        yolo_class_id = cat_id_to_yolo_id[ann['category_id']]
                        for seg in ann['segmentation']:
                            norm_seg = [f"{coord / width if i % 2 == 0 else coord / height:.6f}" for i, coord in enumerate(seg)]
                            f.write(f"{yolo_class_id} {' '.join(norm_seg)}\n")

    unzipped_data_path = os.path.join(temp_unzip_path, "SorghumWeedDataset_Segmentation")
    # Create permanent directories in Drive
    os.makedirs(os.path.join(processed_dataset_path, "images/train"), exist_ok=True); os.makedirs(os.path.join(processed_dataset_path, "images/val"), exist_ok=True); os.makedirs(os.path.join(processed_dataset_path, "images/test"), exist_ok=True)
    # Copy images to their permanent home in Drive
    !cp -n {unzipped_data_path}/Train/*.JPG {processed_dataset_path}/images/train/
    !cp -n {unzipped_data_path}/Validate/*.JPG {processed_dataset_path}/images/val/
    !cp -n {unzipped_data_path}/Test/*.JPG {processed_dataset_path}/images/test/
    # Convert labels and save them to their permanent home in Drive
    convert_coco_to_yolo_segmentation(os.path.join(unzipped_data_path, "Train", "TrainSorghumWeed_coco.json"), os.path.join(processed_dataset_path, "labels/train"))
    convert_coco_to_yolo_segmentation(os.path.join(unzipped_data_path, "Validate", "ValidateSorghumWeed_coco.json"), os.path.join(processed_dataset_path, "labels/val"))
    convert_coco_to_yolo_segmentation(os.path.join(unzipped_data_path, "Test", "TestSorghumWeed_coco.json"), os.path.join(processed_dataset_path, "labels/test"))

    # 5. Create the permanent YAML file in Drive
    print("\nSTEP 4: Creating permanent dataset.yaml file in Drive...")
    class_names = ['Sorghum', 'Grasses', 'Broad-leaf weeds']
    dataset_config = {'path': processed_dataset_path, 'train': 'images/train', 'val': 'images/val', 'test': 'images/test', 'nc': len(class_names), 'names': class_names}
    with open(os.path.join(processed_dataset_path, 'dataset.yaml'), 'w') as file:
        yaml.dump(dataset_config, file, default_flow_style=False)

    # 6. Clean up temporary files
    print("\nSTEP 5: Cleaning up temporary files...")
    !rm -rf {temp_unzip_path}

    print("\n\n✅🚀 PERMANENT SETUP COMPLETE! Your processed dataset and YAML file are now saved in your Google Drive.")

STEP 1: Mounting Google Drive...
Mounted at /content/drive

STEP 2: Unzipping source data from /content/drive/MyDrive/Sorghum_Project/source_dataset/SorghumWeedDataset_Segmentation/SorghumWeedDataset_Segmentation.zip...
✅ Unzip complete.

STEP 3: Converting annotations and saving to Drive...
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


  -> Processing Train: 100%|██████████| 202/202 [00:02<00:00, 95.90it/s] 


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


  -> Processing Validate: 100%|██████████| 25/25 [00:00<00:00, 100.88it/s]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


  -> Processing Test: 100%|██████████| 25/25 [00:00<00:00, 76.48it/s]



STEP 4: Creating permanent dataset.yaml file in Drive...

STEP 5: Cleaning up temporary files...


✅🚀 PERMANENT SETUP COMPLETE! Your processed dataset and YAML file are now saved in your Google Drive.


In [ ]:
# Install the library
!pip install -q ultralytics

# Start the training
!yolo task=segment mode=train \
      model='yolov8s-seg.pt' \
      data='/content/drive/MyDrive/Sorghum_Project/yolo_dataset/dataset.yaml' \
      epochs=50 \
      project='/content/drive/MyDrive/Sorghum_Project/training_runs' \
      name='run_1_50_epochs'

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Sorghum_Project/yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run_1_50_epochs, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True,

In [ ]:
!yolo task=segment mode=train \
      model='yolov8s-seg.pt' \
      data='/content/drive/MyDrive/Sorghum_Project/yolo_dataset/dataset.yaml' \
      epochs=50 \
      imgsz=640 \
      batch=8 \
      project='/content/drive/MyDrive/Sorghum_Project/training_runs' \
      name='run_2_with_augmentations_fresh' \
      augment=True \
      degrees=20 \
      translate=0.1 \
      scale=0.1 \
      hsv_s=0.5 \
      hsv_v=0.5 \
      copy_paste=0.1

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Sorghum_Project/yolo_dataset/dataset.yaml, degrees=20, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.5, imgs

In [ ]:
# This command lists the contents of your results folder
!ls -l /content/drive/MyDrive/Sorghum_Project/training_runs/run_1_50_epochs/

total 9196
-rw------- 1 root root    1684 Sep 22 05:37 args.yaml
-rw------- 1 root root  197063 Sep 22 06:16 BoxF1_curve.png
-rw------- 1 root root  173575 Sep 22 06:16 BoxP_curve.png
-rw------- 1 root root  127567 Sep 22 06:16 BoxPR_curve.png
-rw------- 1 root root  164155 Sep 22 06:16 BoxR_curve.png
-rw------- 1 root root  166001 Sep 22 06:17 confusion_matrix_normalized.png
-rw------- 1 root root  147946 Sep 22 06:17 confusion_matrix.png
-rw------- 1 root root  150249 Sep 22 05:38 labels.jpg
-rw------- 1 root root  197270 Sep 22 06:16 MaskF1_curve.png
-rw------- 1 root root  171972 Sep 22 06:16 MaskP_curve.png
-rw------- 1 root root  127679 Sep 22 06:16 MaskPR_curve.png
-rw------- 1 root root  163405 Sep 22 06:16 MaskR_curve.png
-rw------- 1 root root    8905 Sep 22 06:11 results.csv
-rw------- 1 root root     181 Sep 22 05:52 results.gsheet
-rw------- 1 root root  462112 Sep 22 06:17 results.png
-rw------- 1 root root 1044155 Sep 22 05:38 train_batch0.jpg
-rw------- 1 root root  969

In [ ]:
!yolo task=segment mode=train \
      model='yolov8s-seg.pt' \
      data='/content/drive/MyDrive/Sorghum_Project/yolo_dataset/dataset.yaml' \
      epochs=50 \
      imgsz=640 \
      batch=8 \
      project='/content/drive/MyDrive/Sorghum_Project/training_runs' \
      name='run_2_with_augmentations' \
      augment=True \
      degrees=20 \
      translate=0.1 \
      scale=0.1 \
      hsv_s=0.5 \
      hsv_v=0.5 \
      copy_paste=0.1

In [ ]:
import os
from ultralytics import YOLO
from google.colab import drive

# Ensure Drive is mounted (if not already)
drive.mount('/content/drive', force_remount=True)

# Define paths for your project
project_drive_path = "/content/drive/MyDrive/Sorghum_Project"
data_yaml_path = os.path.join(project_drive_path, 'yolo_dataset/dataset.yaml')
tuning_runs_path = os.path.join(project_drive_path, 'tuning_runs')
run_name = 'yolov8s_tuning_python_api'

print("Loading standard YOLOv8s-seg model for tuning.")
# Load the standard YOLOv8s-seg model
model = YOLO('yolov8s-seg.pt')

print(f"Starting hyperparameter tuning run: {run_name}")
# Tune the model with all your desired arguments
# iterations will be determined by the 'n' parameter in the tune method
model.tune(
    data=data_yaml_path,
    epochs=50,  # Each iteration trains for this many epochs
    imgsz=640,
    batch=8,
    iterations=100, # This is the number of hyperparameter combinations to try
    project=tuning_runs_path,
    name=run_name,
    # You can add other tuning parameters here if desired, like plotting=True etc.
    # The 'tune' method handles its own augmentations by default.
)

print(f"\n✅ Hyperparameter tuning complete! Results saved to: {os.path.join(tuning_runs_path, run_name)}")

✅ Success: CBAM module has been successfully registered with YOLO.
Mounted at /content/drive
An error occurred: 'CBAM'


In [ ]:
#hyperparameter tuning

import os
from ultralytics import YOLO
from google.colab import drive

# Ensure Drive is mounted (if not already)
drive.mount('/content/drive', force_remount=True)

# Define paths for your project
project_drive_path = "/content/drive/MyDrive/Sorghum_Project"
data_yaml_path = os.path.join(project_drive_path, 'yolo_dataset/dataset.yaml')
tuning_runs_path = os.path.join(project_drive_path, 'tuning_runs')
run_name = 'yolov8s_tuning_python_api'

print("Loading standard YOLOv8s-seg model for tuning.")
# Load the standard YOLOv8s-seg model
model = YOLO('yolov8s-seg.pt')

print(f"Starting hyperparameter tuning run: {run_name}")
# Tune the model with all your desired arguments
# iterations will be determined by the 'n' parameter in the tune method
model.tune(
    data=data_yaml_path,
    epochs=50,  # Each iteration trains for this many epochs
    imgsz=640,
    batch=8,
    iterations=100, # This is the number of hyperparameter combinations to try
    project=tuning_runs_path,
    name=run_name,
    # You can add other tuning parameters here if desired, like plotting=True etc.
    # The 'tune' method handles its own augmentations by default.
)

print(f"\n✅ Hyperparameter tuning complete! Results saved to: {os.path.join(tuning_runs_path, run_name)}")

Mounted at /content/drive
Loading standard YOLOv8s-seg model for tuning.
Starting hyperparameter tuning run: yolov8s_tuning_python_api
Tuner: Initialized Tuner instance with 'tune_dir=/content/drive/MyDrive/Sorghum_Project/tuning_runs/yolov8s_tuning_python_api'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/100 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0, 'close_mosaic': 10}


KeyboardInterrupt: 